## DenseNetwork 구현 및 학습

In [1]:
import tensorflow as tf
import numpy as np

## 데이터셋 준비

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## DenseUnit 구현

In [3]:
class DenseUnit(tf.keras.Model):
    def __init__(self, filter_out, kernel_size):
        super(DenseUnit, self).__init__()
        
        # Pre-Activation 구조
        self.bn = tf.keras.layers.BatchNormalization()
        self.conv = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        # Concatenate
        self.concat = tf.keras.layers.Concatenate()

    def call(self, x, training=False, mask=None): # x: (Batch, H, W, Ch_in)
        h = self.bn(x, training=training)
        h = tf.nn.relu(h)
        h = self.conv(h)                          # h: (Batch, H, W, filter_out)
        
        return self.concat([x, h])                # (Batch, H, W, (Ch_in + filter_out))

## DenseLayer 구현

In [4]:
class DenseLayer(tf.keras.Model):
    def __init__(self, num_unit, growth_rate, kernel_size):
        super(DenseLayer, self).__init__()
        
        self.sequence = list()
        for idx in range(num_unit):
            self.sequence.append(DenseUnit(growth_rate, kernel_size))

    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training=training)
        return x

## Transition Layer 구현
- maxpooling 시 필요. channel 수를 줄이는 효과

In [5]:
class TransitionLayer(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(TransitionLayer, self).__init__()
        
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.pool = tf.keras.layers.MaxPool2D()

    def call(self, x, training=False, mask=None):
        x = self.conv(x)
        return self.pool(x)

## 모델 정의

In [6]:
class DenseNet(tf.keras.Model):
    def __init__(self):
        super(DenseNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(8, (3, 3), padding='same', activation='relu') # 28x28x8
        
        # num_unit, growth_rate, kernel_size
        self.dl1 = DenseLayer(2, 4, (3, 3))    # 28x28x16   (8 + 2*4)
        self.tr1 = TransitionLayer(16, (3, 3)) # 14x14x16
        
        self.dl2 = DenseLayer(2, 8, (3, 3))    # 14x14x32   (16 + 2*8)
        self.tr2 = TransitionLayer(32, (3, 3)) # 7x7x32
        
        self.dl3 = DenseLayer(2, 16, (3, 3))   # 7x7x64     (32 + 2*16)
        
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')       

    def call(self, x, training=False, mask=None):
        x = self.conv1(x)
        
        x = self.dl1(x, training=training)
        x = self.tr1(x)
        
        x = self.dl2(x, training=training)
        x = self.tr2(x)
        
        x = self.dl3(x, training=training)
        
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)        

## 학습, 테스트 루프 정의

In [7]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [8]:
# Create model
model = DenseNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [9]:
EPOCHS = 10

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {:.3f}, Accuracy: {:.3f}, Test Loss: {:.3f}, Test Accuracy: {:.3f}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result(),
                          test_loss.result(),
                          test_accuracy.result()))
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.110, Accuracy: 0.968, Test Loss: 0.047, Test Accuracy: 0.985
Epoch 2, Loss: 0.059, Accuracy: 0.983, Test Loss: 0.051, Test Accuracy: 0.984
Epoch 3, Loss: 0.045, Accuracy: 0.988, Test Loss: 0.055, Test Accuracy: 0.984
Epoch 4, Loss: 0.038, Accuracy: 0.989, Test Loss: 0.050, Test Accuracy: 0.986
Epoch 5, Loss: 0.037, Accuracy: 0.990, Test Loss: 0.042, Test Accuracy: 0.989
Epoch 6, Loss: 0.032, Accuracy: 0.991, Test Loss: 0.049, Test Accuracy: 0.987
Epoch 7, Loss: 0.027, Accuracy: 0.993, Test Loss: 0.059, Test Accuracy: 0.989
Epoch 8, Loss: 0.030, Accuracy: 0.992, Test Loss: 0.048, Test Accuracy: 0.990
Epoch 9, Loss: 0.024, Accuracy: 0.994, Test Loss: 0.070, Test Accuracy: 0.986
Epoch 10, Loss: 0.024, Accuracy: 0.994, Test Loss: 0.053, Test Accuracy: 0.989
